You need to make sure QuPath.exe is in usr/bin or else [check Paquo documentation](https://paquo.readthedocs.io/en/latest/configuration.html#configuration) about making it accessible to Python. Make sure to pip install paquo and rasterio. Note: the measurements 1, 2, 3 below are all random placeholders for now. This takes about 20 seconds.

In [ ]:
from paquo.projects import QuPathProject
import numpy as np
from tifffile import imread
import rasterio
from rasterio import features
import shapely
from shapely.geometry import Point, Polygon

In [ ]:

def mask_to_polygons_layer(mask):
	all_polygons = []
	for shape, value in features.shapes(mask.astype(np.int16), mask=(mask >0), transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0)):
		all_polygons.append(shapely.geometry.shape(shape))
	return all_polygons

In [ ]:

mask = imread("./RAM13_003-Brain_131199-2.ome_mask.tif")
polygons = mask_to_polygons_layer(mask)
test_measurements = np.random.randint(0, 100, size=(len(polygons), 5)).astype(float)

In [ ]:

with QuPathProject('./folder_name/project.qpproj', mode = "x") as qp:
	entry = qp.add_image('RAM13_003-Brain_131199-2.ome.tif')
	hierarchy = entry.hierarchy
	with hierarchy.no_autoflush():
	for i, polygon in enumerate(polygons):
		detection_measurements = {
		"Measurement 1": test_measurements[i, 0],
		"Measurement 2": test_measurements[i, 1],
		"Measurement 3": test_measurements[i, 2],
		}
	detection = hierarchy.add_detection(polygon, measurements = detection_measurements)
	detection.name = f"Detection {i}"